In [1]:
import numpy as np
from skimage import io
from matplotlib import pyplot as plt

In [29]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
from PIL import Image
import os
from skimage import filters
import shutil


%matplotlib inline

In [7]:
def get_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [8]:
def remove_green (image_path):
  img = get_image(image_path)
  _, threshold = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
  
  for row in range(len(threshold)):
    for col in range(len(threshold[row])):
      if (list(threshold[row][col]) == [0,255,0] ): #or list(threshold[row][col]) == [255,255,0]
        threshold[row][col] = [0,0,0]

  plt.subplot(1,2,1),plt.imshow(img)
  plt.subplot(1,2,2),plt.imshow(threshold)

  return threshold

In [52]:
def binary_mask_image(image_path):

    path = '102flowers/'+image_path
    image_no_green = Image.fromarray((remove_green(path)))
    image_no_green.save('new_images/{}'.format(image_path))
    image = cv2.imread('new_images/{}'.format(image_path),0)
    os.remove('new_images/{}'.format(image_path))

    _,threshold_image = cv2.threshold(image,0,255,cv2.THRESH_BINARY)
    im_smooth = filters.rank.percentile(threshold_image, np.ones((21,21)), p0=0.5)

    original = get_image(path)
    mask = im_smooth / 255
    mask = mask.reshape(*mask.shape, 1)
    masked_image = original * mask

    image_mask = Image.fromarray(masked_image.astype(np.uint8))
    print("i'm saving masked image...")
    image_mask.save("mask/{}".format(image_path))
    
    plt.imshow(image_mask, cmap = 'gray')

In [2]:
list_files = os.listdir('D:/UNIVERSITA_2/Magistrale/AdvancedML/PROGETTO/102flowers')

In [ ]:
for image in list_files:
    binary_mask_image(image)

In [ ]:
def rotate_image (image_path):
  path = '102flowers/'+image_path
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  img_180 = img.rotate(180)
  img_180.save('rotated/{}'.format(image_path))

list_files = os.listdir('102flowers/')
for image in list_files:
    print (image)
    rotate_image(image)

In [ ]:
def mask_grabcut (image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = np.zeros(img.shape[:2],np.uint8)

    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (5,5,224,224)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,3,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img = img*mask2[:,:,np.newaxis]

    return img

In [ ]:
def mask_image(image_path):
    img1 = mask_grabcut('102flowers/{}'.format(image_path))
    img_180 = mask_grabcut('rotated/{}'.format(image_path))
    img_180= Image.fromarray(img_180)
    img_180 = img_180.rotate(180)

    overlap  = img1*img_180
    overlap = cv2.cvtColor(overlap, cv2.COLOR_RGB2GRAY)
    im_smooth = filters.rank.percentile(overlap, np.ones((2,2)), p0=0.5)

    _,threshold_image = cv2.threshold(overlap,0,255,cv2.THRESH_BINARY)

    mask = threshold_image / 255
    mask = mask.reshape(*mask.shape,1)
    masked_image = img1 * mask

    image_mask = Image.fromarray(masked_image.astype(np.uint8))
    print("i'm saving masked image...")
    image_mask.save("mask_NEW/{}".format(image_path))

In [ ]:
list_files = os.listdir('102flowers/')
for image in list_files:
    print (image)
    mask_image(image)

In [ ]:
list_files_tot = os.listdir('D:/UNIVERSITA_2/Magistrale/AdvancedML/PROGETTO/102flowers')
list_files_mask = os.listdir('D:/UNIVERSITA_2/Magistrale/AdvancedML/PROGETTO/mask_NEW')
list_remove = ['image_00503.jpg','image_02602.jpg','image_03180.jpg','image_03917.jpg','image_04814.jpg','image_05841.jpg','image_5844.jpg','image_06975.jpg','image_06976.jpg','image_06999.jpg']

In [ ]:
import shutil
import cv2
from PIL import Image

In [ ]:
source = 'mask_tot/'
dest = 'mask_NEW/'
for image in sorted(list_files_tot):
    if image not in list_files_mask:
        #if image not in list_remove:
        shutil.copyfile(source+image, dest+image)

In [ ]:
source = 'mask_NEW/'
list_files_tot = os.listdir('D:/UNIVERSITA_2/Magistrale/AdvancedML/PROGETTO/mask_NEW')

for i in range(len(list_files_tot)):
    if list_files_tot[i] in list_remove:
        path_curr = source + list_files_tot[i]
        path_prec = source + list_files_tot[i-1]
        
        img = cv2.imread(path_prec)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_90 = img.rotate(90)
        img_90.save(path_curr)